# 7. Building a Project Manager (PM) Agent for VM Migration in Python

In this notebook, you will learn how to build and deploy a Project Manager (PM) agent designed to manage the execution of a Virtual Machine (VM) migration plan. This agent will coordinate various tasks, monitor progress, and dynamically adapt the plan based on feedback from different engineering agents. By the end of this notebook, you will understand how to:

- Set up and configure a model to support the PM agent's decision-making process.
- Define and manage the state for the PM agent to track task progress and dependencies.
- Implement and customize the PM agent to handle specific responsibilities within the VM migration workflow.

## 1. Setting up the Environment

Before we can communicate with the LLM, let’s install any required libraries and ensure our environment is ready.

In [60]:
%pip install requests jsonschema tenacity

Note: you may need to restart the kernel to use updated packages.


These packages are used for:

- **requests:** Making HTTP requests to interact with models.
- **jsonschema:** Validating the structure of the agent's output.
- **tenacity:** Handling retries in case of errors when communicating with the model.

### Basic Agent Data Handling
Next, we'll introduce a simple way to handle data for our agent. In this case, we'll create a function that allows us to load and manage the agent's data. This data might include things like the agent's responsibilities, the tasks it needs to perform, and other relevant information.

Let's assume the data is stored in a YAML file (a common format for configuration files), and we'll write a function to load this data:

In [61]:
import yaml
from typing import Dict, Any


def load_agent_descriptions(description_file: str) -> Dict[str, Any]:
    """
    Load the agent descriptions from a YAML file.
    """
    try:
        with open(description_file, "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"Description file '{description_file}' not found.")


# Example usage:
agents_description = load_agent_descriptions("agents.yaml")
print("Loaded agent descriptions:", agents_description)

Loaded agent descriptions: {'agents': [{'name': 'Planner Agent', 'role': 'Planner Agent', 'responsibilities': ['Creates a comprehensive Migration Plan based on the tutorial.', 'Identifies key steps, target VMs, and source/target providers.', 'Coordinates and structures the plan for execution by other agents.']}, {'name': 'Project Manager (PM) Agent', 'role': 'Project Manager Agent', 'responsibilities': ['Manages the breakdown of tasks for the migration process.', 'Oversees task execution and ensures agents are working in coordination.', 'Ensures timelines are followed and adjusts the plan as necessary.', 'Communicates with all agents to ensure smooth task progression and resolve bottlenecks.']}, {'name': 'vSphere Engineer Agent', 'role': 'vSphere Engineer Agent', 'responsibilities': ['Handles VM identification and configuration within the vSphere environment.', 'Identifies VMs to migrate based on the tutorial instructions.', 'Manages and configures the Migration Toolkit for Virtualizat

## 2. Configuring a Simple Model

In this section, we configure the machine learning model that our agent will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16"), allowing the agent to handle tasks such as listing VMs and retrieving details.

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables our agent to perform model-based tasks using the provided configuration.

In [62]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:latest")

## 3. System Prompt for the PM Agent

When the PM agent receives a query, the system prompt shapes how it processes the input and generates its output. This includes breaking down tasks, assigning them to the correct agents, and handling feedback—all within the structure provided by the prompt.

In [63]:
DEFAULT_SYS_PM_PROMPT = """
system

Environment: ipython  
Cutting Knowledge Date: December 2023  
Today Date: {datetime}  

You are the **Project Manager (PM) Agent** responsible for transforming the **Migration Plan Document (MPD)** into an actionable execution plan. Your mission is to ensure the smooth execution of migration tasks by coordinating agents (OCP Engineer, vSphere Engineer, Cleanup) and monitoring progress from start to finish.

---

### Feedback Handling:
If any task encounters issues or feedback, adapt the execution plan dynamically to accommodate the changes. Ensure that all agents are informed accordingly and that tasks are adjusted based on feedback. Here is the feedback received:
Feedback:

{feedback}

---

### Agents Description:
{agents_description}


---

### Guidelines:

1. **Task Status Management:** Monitor feedback from agents (e.g., OCP Engineer, vSphere Engineer) and update the status of existing tasks. Only create new tasks if explicitly required by new information or feedback.
2. **Transform MPD into Tasks:** Break down the MPD into clear, actionable tasks with unique IDs, names, descriptions, assigned agents, dependencies, and acceptance criteria.
3. **Assign Agents:** Assign each task to the correct agent based on the task’s requirements and the agents’ roles.
4. **Track Status:** Continuously monitor task progress and update task statuses as `"pending"`, `"in_progress"`, `"completed"`, or `"failed"`.
5. **Handle Dependencies:** Ensure that tasks only begin after all their dependencies are completed.
6. **Facilitate Communication:** Ensure smooth communication between agents to resolve task dependencies or blockers.
7. **Define Acceptance Criteria:** Ensure each task has clear acceptance criteria, and update task statuses based on the completion of those criteria.

---

### Output Format (Task Structure):

Your response should return a task list in the following format. This task list is central to ensuring that all migration tasks are tracked and executed correctly:

{{
    "tasks": [
        {{
            "task_id": "string",  # Unique identifier for the task.
            "task_name": "string",  # The short name of the task (e.g., "Create Migration Plan").
            "task_description": "string",  # A detailed description of the task's actions.
            "agent": "string",  # The agent responsible for executing the task (must be one of: "ocp_engineer", "vsphere_engineer", "cleanup").
            "status": "pending",  # The current status of the task ("pending", "in_progress", "completed", "failed").
            "dependencies": ["array"],  # Task IDs that must be completed before this task starts.
            "acceptance_criteria": "string",  # The criteria for determining task success (e.g., "Migration plan created and validated").
            "tool_to_use": "string or null",  # The tool that the agent should use to execute the task. This may be null if no tool is required.
            "provided_inputs": {{
                "key": "string | array | null"  # Input data needed for this task, such as VM names or configurations.
            }}
        }}
    ]
}}

---

### Example of a Task List:

{{
    "tasks": [
        {{
            "task_id": "task_001",
            "task_name": "Retrieve VM Details (Database)",
            "task_description": "The vSphere engineer will retrieve the details of the virtual machine named 'database'.",
            "agent": "vsphere_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "VM details retrieved successfully.",
            "tool_to_use": "retrieve_vm_details_tool"
        }},
        {{
            "task_id": "task_002",
            "task_name": "Create Migration Plan",
            "task_description": "Create a migration plan for the specified virtual machines.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "Migration plan created and validated.",
            "tool_to_use": "create_migration_plan_tool",
            "provided_inputs": {{
                "vm_names": ["vm1", "vm2", "vm3"],
                "plan_name": "database-plan"
            }}
        }},
        {{
            "task_id": "task_003",
            "task_name": "Start Migration",
            "task_description": "Start the migration process using the migration plan.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": ["task_001"],
            "acceptance_criteria": "Migration process successfully started.",
            "tool_to_use": "start_migration_tool",
            "provided_inputs": {{
                "plan_name": "database-plan"
            }}
        }}
    ]
}}


---

Remember:
- **Do not change the original task plan** unless explicitly required by new information, dependencies, or feedback.
- Always prioritize updating existing tasks based on feedback before creating new tasks.
- Ensure that the original tasks are followed as closely as possible to avoid unnecessary changes in the execution plan.
- Ensure tasks are marked as complete once their acceptance criteria are met.
- Maintain the JSON format and ensure all fields are filled out correctly, including the `tool_to_use` field with the appropriate tool name.
"""

In [64]:
from utils.general.helpers import get_current_utc_datetime


def write_pm_prompt(
    feedback_value: str = "",
    agents_description: str = agents_description,
) -> str:
    return DEFAULT_SYS_PM_PROMPT.format(
        agents_description=agents_description,
        feedback=feedback_value,
        datetime=get_current_utc_datetime(),
    )

## 4. Defining the PM Agent's State and Role

The **AgentGraphState** class and the **Agent** class are integral parts of the PM agent's structure. The agent's state keeps track of messages and feedback, which are crucial for managing the progress of tasks and addressing issues dynamically as they arise during the VM migration.

### Agent State with `AgentGraphState`
We start by defining the **AgentGraphState** class. This class inherits from `TypedDict`, which allows us to define typed dictionaries in Python. The dictionary contains two key properties:
- `response`: A list of messages that store the responses generated by the PM agent.
- `feedback`: A list of messages related to the feedback received from agents.

Both of these properties are annotated with `add_messages`, a utility that helps manage message flows in the agent's graph of communications.

In [65]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any

class AgentGraphState(TypedDict):
    response: Annotated[list, add_messages]
    feedback: Annotated[list, add_messages]


### Creating the PM Agent

Next, we instantiate the **PM Agent** by creating an instance of the **Agent** class. This agent is responsible for managing the VM migration tasks, coordinating the engineers, and processing the feedback.

- The `state` argument is initialized with the **AgentGraphState** dictionary to keep track of the agent’s responses and feedback.
- The `role` argument is set to `"PM_AGENT"`, defining this agent's role as the project manager in the system.
- The `ollama_service` is passed in to allow the PM agent to interact with the underlying machine learning model.


In [66]:
from agent.base_agent import Agent

state = AgentGraphState()

pm_agent = Agent(
    state=state,
    role="PM_AGENT",
    ollama_service=ollama_service,
    )

With the PM agent now set up, it will be able to process tasks, handle feedback, and communicate with other agents (like the OCP and vSphere engineers) effectively as part of the migration plan execution.

## 5. Handling User Requests and Generating a Migration Plan

In this section, we simulate a user request for a series of tasks that will be executed by the **PM Agent**. The tasks involve a vSphere engineer and an OCP engineer working together to retrieve virtual machine (VM) details, create a migration plan, and finally start the migration plan.

### Simulating a User Request
We define a `user_req` string, which details the tasks that the user wants to execute. The user specifies three tasks:

1. The vSphere engineer will retrieve the details of a VM called "database".
2. The OCP engineer will create a migration plan for the "database" VM, with the plan named "database-plan".
3. The OCP engineer will start the migration plan, "database-plan".

This user request is passed into the PM agent for processing.


### Processing the User Request with the PM Agent

The PM agent processes the request using the `work()` method. This method takes two arguments:
1. **`user_request`**: The user-defined request with the tasks outlined above.
2. **`sys_prompt`**: The system prompt generated by the **`write_pm_prompt()`** function, which sets the guidelines and agents' descriptions for the PM agent to follow.

The response from the agent includes the generated task plan, which is stored in the `PM_AGENT_response` field of the response.

In [67]:
user_req = """These are three tasks.  
First task: The vSphere engineer will Retrieve the VM details of the vm called database. 
Don't forget the second Task: The OCP Engineer will Create a migration plan (vm_name: database) called 'database-plan'. 
Third task: The Ocp engineer will start the 'database-plan'. 
Just do this. Treat this as THREE different tasks."""

response =pm_agent.work(user_request=user_req, sys_prompt=write_pm_prompt())

#### Viewing the Generated Migration Plan

The generated task plan is extracted from the `response` object, and its content is printed to provide insight into the tasks assigned to each agent.

In [68]:
plan = response["PM_AGENT_response"].content
print(plan)

{
    "tasks": [
        {
            "task_id": "task_001",
            "task_name": "Retrieve VM Details (Database)",
            "task_description": "The vSphere engineer will retrieve the details of the virtual machine named 'database'.",
            "agent": "vsphere_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "VM details retrieved successfully.",
            "tool_to_use": null,
            "provided_inputs": {
                "vm_name": "database"
            }
        },
        {
            "task_id": "task_002",
            "task_name": "Create Migration Plan (Database)",
            "task_description": "The OCP Engineer will create a migration plan for the specified virtual machine called 'database' and named it 'database-plan'.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": [
                "task_001"
            ],
            "acceptance_criteria": "M

## 6. Validating Agent Output

To maintain the reliability and consistency of the PM agent, it's crucial to validate the output generated by the agent. In this step, we define a schema that outlines the expected structure of the tasks created by the PM agent. This schema includes fields like `task_id`, `task_name`, `task_description`, `agent`, `status`, and others, ensuring that each task is well-defined and follows a standardized format.

In [69]:
from jsonschema import validate, ValidationError

# Example schema for validating the agent's output
output_schema = {
    "type": "object",
    "properties": {
        "tasks": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "task_id": {
                        "type": "string",  # Unique identifier for the task.
                    },
                    "task_name": {
                        "type": "string",  # Short name of the task (e.g., "Validate VMware Access").
                    },
                    "task_description": {
                        "type": "string",  # Detailed description of the task to be executed.
                    },
                    "agent": {
                        "type": "string",
                        "enum": [
                            "architect",
                            "ocp_engineer",
                            "vsphere_engineer",
                            "networking",
                            "reviewer",
                            "cleanup",
                        ],  # Agent responsible for executing the task.
                    },
                    "status": {
                        "type": "string",
                        "enum": [
                            "pending",
                            "in_progress",
                            "completed",
                            "failed",
                        ],  # Current status of the task.
                    },
                    "dependencies": {
                        "type": "array",
                        "items": {
                            "type": "string",  # Task IDs that must be completed before this task can start.
                        },
                        "default": [],  # Default is an empty array if there are no dependencies.
                    },
                    "acceptance_criteria": {
                        "type": "string",  # Criteria that must be met to consider the task successfully completed.
                    },
                    "tool_to_use": {
                        "type": [
                            "string",
                            "null",
                        ],  # The tool the agent should use, or null if no tool is required.
                        "default": None,  # Default is None if no tool is specified.
                    },
                    "provided_inputs": {
                        "type": "object",  # Input data needed for this task.
                        "additionalProperties": {
                            "type": ["null", "string", "array"],
                            "items": {"type": "string"},
                        },
                        "default": {},  # Default is an empty object if no inputs are provided.
                    },
                },
                "required": [
                    "task_id",
                    "task_name",
                    "task_description",
                    "agent",
                    "status",
                    "acceptance_criteria",
                ],  # Dependencies, tool_to_use, and provided_inputs are optional.
            },
        }
    },
    "required": ["tasks"],
}

### Validation Process:
- **Validation Function (`validate_agent_output`)**: This function checks the agent's output against the predefined schema. If the output conforms to the schema, it is marked as validated. Otherwise, it returns feedback indicating the issues, which can then be used to adjust the agent's behavior.

Validating the output ensures that the tasks generated by the PM agent are actionable and correctly formatted, which is essential for the smooth execution of the VM migration plan.

In [70]:
from langchain_core.messages import SystemMessage


def validate_agent_output(plan: str, schema: dict = output_schema):
    """
    Validate the agent's output against the provided schema.
    """
    json_plan = json.loads(plan)
    try:
        validate(instance=json_plan, schema=schema)
        return True
    except ValidationError as e:
        return False

In [71]:
def save_json_to_file(json_plan, filename=""):
    try:
        # Save the content of json_plan to a file
        with open(filename, "w") as file:
            json.dump(json_plan, file, indent=4)
        print(f"JSON saved successfully to {filename}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")


if validate_agent_output(plan, output_schema):
    print("The plan is validated!")
    save_json_to_file(plan, "data/json_plan_output.json")
else:
    print("The plan is not validated!")

The plan is validated!
JSON saved successfully to data/json_plan_output.json
